In [1]:
import torch
import string
import glob
import unicodedata
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from customlstm import CustomLSTM
from dataprep import Serialdata

In [2]:
class InferSerialData(Dataset):
    def __init__(self):
        self.all_chars = string.ascii_letters + " .,;'"
        self.num_char = len(self.all_chars)
        self.all_chars_dict = {v: k for k, v in enumerate(self.all_chars)}
        self.all_instances = []
    
    def unicodeToAscii(self, s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn'
            and c in self.all_chars
        )
    
    def convert(self, line):
        
        temp = map(lambda a: self.all_chars_dict.get(a, self.num_char), self.unicodeToAscii(line))
        return torch.tensor(list(temp))

In [3]:
infd = InferSerialData()

In [4]:
infd.convert("amuletos")

tensor([ 0, 12, 20, 11,  4, 19, 14, 18])

In [5]:
# load lstm model for encoding
# sd = Serialdata('combined.tsv')

In [6]:
# sd.num_char

In [7]:
model = CustomLSTM(58, 256, num_layers=2).to('cuda')

{'air': 0, 'art': 1, 'drink': 2, 'lw': 3, 'vw': 4}


In [8]:
model.load_state_dict(torch.load('lstm_model.pt'))

<All keys matched successfully>

In [11]:
x, y = model(torch.tensor(infd.convert("amuletos")).unsqueeze(0).to('cuda'))

C:\Users\Rah12937\AppData\Local\Temp\ipykernel_34112\2130371112.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x, y = model(torch.tensor(infd.convert("amuletos")).unsqueeze(0).to('cuda'))


In [57]:
p = torch.tensor(infd.convert(",")).unsqueeze(0)

C:\Users\Rah12937\AppData\Local\Temp\ipykernel_34112\369803096.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p = torch.tensor(infd.convert(",")).unsqueeze(0)


In [58]:
p

tensor([[54]])

In [59]:
list(model.named_children())[0][1](p.to('cuda')).shape

torch.Size([1, 1, 58])

In [35]:
y[0,:, :].shape

torch.Size([1, 256])

In [42]:
torch.mean(y, dim=0, keepdim=True).shape

torch.Size([1, 1, 256])

In [49]:
k = torch.randn([1,1,768]).to('cuda')

In [51]:
torch.cat([torch.mean(y, dim=0, keepdim=True),k], dim=2).shape

torch.Size([1, 1, 1024])

In [67]:
list(model.named_children())[0][1](torch.tensor(57).cuda())

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       grad_fn=<EmbeddingBackward0>)

In [72]:
torch.mean(y, dim=0, keepdim=True)

tensor([[[-1.4850e-01,  1.1388e-01, -8.1643e-02,  3.9785e-02, -2.1584e-01,
          -7.8105e-02,  1.8852e-01, -4.2600e-01, -9.7662e-02, -6.5057e-03,
          -1.1884e-01,  1.4623e-02, -1.0047e-01,  5.3203e-03, -1.3843e-01,
          -2.0081e-02, -8.4136e-02, -3.2791e-01,  3.9768e-01,  1.1855e-01,
           3.2262e-01, -2.5397e-02,  8.9044e-03,  2.3012e-01,  5.9491e-04,
          -8.0250e-02, -1.1846e-01, -7.2692e-02, -8.9033e-02,  6.3456e-03,
          -5.1993e-02,  1.1373e-02,  3.2933e-01,  1.3531e-01, -1.3157e-02,
          -3.4644e-01,  3.9366e-01,  1.8917e-01, -1.3925e-02, -1.3347e-02,
           5.2372e-02, -7.7973e-02,  1.3047e-01,  3.8390e-01,  3.3486e-02,
           2.7461e-02, -1.8372e-01, -1.5298e-02, -1.3282e-01,  4.9432e-02,
           3.0092e-01,  1.9120e-01,  2.6052e-01,  3.9349e-02,  3.7179e-02,
           4.3459e-02,  1.4674e-02,  2.7423e-01,  4.3230e-02,  7.2920e-02,
           1.7448e-01, -2.7882e-01,  2.7586e-01, -1.1346e-01,  1.7647e-01,
          -8.8797e-02, -6

In [74]:
torch.stack([torch.mean(y, dim=0, keepdim=True),torch.mean(y, dim=0, keepdim=True)], dim=2).shape

torch.Size([1, 1, 2, 256])

In [77]:
torch.tensor([]).size()[0]

torch.Size([0])